# 자동차 이미지들을 학습 및 테스트합니다.

훈련 데이터는 각 240개씩 총 960개로 구성되어있고,
시험 데이터는 각 60개씩 총 240개로 구성되어있습니다.

차종은 세단, 스포츠카, SUV, 픽업트럭으로 네가지를 구분하였고 종류별 숫자 코드는 다음과 같습니다.
0 = sedan, 1 = sportscar, 2 = suv, 3 = truck

In [1]:
import os, random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


    Namin Neural Network
    -----------------
    conv1 - relu1 - pool1 - bn1
    conv2 - relu2 - pool2 - bn2
    conv3 - relu3 - pool3 - bn3
    affine - softmax
    
    Parameters
    ----------
    Input Layer(Input Size): 100*100*3
    First Layer: Conv1(3EA, 3*3*3, Strides=3, Padding=1) - ReLU1
                - 34*34*3 - Pool1(2*2, Strides=1) - 33*33*3 - Bn1
    Second Layer: Conv2(6EA, 6*6*3, Strides=1, Padding=VALID) - ReLU2
                - 28*28*6 - Pool2(2*2, Strides=2) - 14*14*6 - Bn2
    Third Layer: Conv3(9EA, 3*3*6, Strides=3, Padding=2) - ReLU3
                - 6*6*9 - Pool3(2*2, Strides=2) - 3*3*9 - Bn3
    Output Layer: Affine(W=3*3*9, B=9) - Output Nodes = 4(Sedan, Coupe, SUV, PickupTruck)

In [2]:
# Load to Sedan, Coupe, SUV, PickupTruck Images
trainlist, testlist = [], []
with open('train.txt') as f:
    for line in f:
        tmp = line.strip().split()
        trainlist.append([tmp[0], tmp[1]])
        
with open('test.txt') as f:
    for line in f:
        tmp = line.strip().split()
        testlist.append([tmp[0], tmp[1]])

In [3]:
# Image Preprocessing
IMG_H = 100
IMG_W = 100
IMG_C = 3

def readimg(path):
    img = plt.imread(path)
    return img

def batch(path, batch_size):
    img, label, paths = [], [], []
    for i in range(batch_size):
        img.append(readimg(path[0][0]))
        label.append(int(path[0][1]))
        path.append(path.pop(0))
        
    return img, label

In [4]:
# Neural Network
num_class = 4 # Sedan, Coupe, SUV, PickupTruck

with tf.Graph().as_default() as g:
    X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C]) # Input Layer, X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C])
    Y = tf.placeholder(tf.int32, [None]) # Y = tf.placeholder(tf.int32, [None])
    
    with tf.variable_scope('CNN'):
        # 1st Layer(Conv1 - relu1 - maxpool1 - bn1) = 33*33*3
        conv1 = tf.layers.conv2d(X, 3, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(conv1, 2, (1, 1), padding='VALID')
        bn1 = tf.compat.v1.layers.batch_normalization(pool1, training=True)
        # 2nd Layer(Conv2 - relu2 - maxpool2 - bn2) = 14*14*6
        conv2 = tf.layers.conv2d(bn1, 6, 6, (1, 1), padding='VALID', activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(conv2, 2, (2, 2), padding='VALID')
        bn2 = tf.compat.v1.layers.batch_normalization(pool2, training=True)
        # 3rd Layer(Conv3 - relu3 - maxpool3 - bn3) = 3*3*9
        conv3 = tf.layers.conv2d(bn2, 9, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(conv3, 2, (2, 2), padding='VALID')
        bn3 = tf.compat.v1.layers.batch_normalization(pool3, training=True)
        # Fully Connected Layer(Affine)
        affine1 = tf.layers.flatten(bn3)
        # Output Layer
        output = tf.layers.dense(affine1, num_class)
        
    # Softmax with Loss
    with tf.variable_scope('Loss'):
        Loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= Y, logits=output))
    
    # Training with Adam    
    train_step = tf.train.AdamOptimizer(0.001).minimize(Loss) 
    saver = tf.train.Saver()
    
    tf.summary.scalar('Epoch-Loss', Loss)
    merged = tf.summary.merge_all()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [5]:
# Size
np.sum([np.product(var.shape) for var in g.get_collection('trainable_variables')]).value

1417

In [6]:
# Setting Batch with Training
batch_size = 1461
epoch = 1000

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('tf_board', sess.graph)
    for i in range(epoch):
        batch_data, batch_label = batch(trainlist, batch_size)     
        _, loss, summary = sess.run([train_step, Loss, merged], feed_dict = {X: batch_data, Y: batch_label})
        print("Epoch:",i+1,"Loss:",loss)
        if i % 10 == 0:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)
        elif i+1 == epoch:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)

Epoch: 1 Loss: 2.0545862
Epoch: 2 Loss: 2.105048
Epoch: 3 Loss: 1.9529419
Epoch: 4 Loss: 1.9840019
Epoch: 5 Loss: 1.8869127
Epoch: 6 Loss: 1.8841138
Epoch: 7 Loss: 1.8211415
Epoch: 8 Loss: 1.80188
Epoch: 9 Loss: 1.7704322
Epoch: 10 Loss: 1.7298973
Epoch: 11 Loss: 1.7282999
Epoch: 12 Loss: 1.6682756
Epoch: 13 Loss: 1.6615964
Epoch: 14 Loss: 1.6272343
Epoch: 15 Loss: 1.6111176
Epoch: 16 Loss: 1.5887657
Epoch: 17 Loss: 1.5664787
Epoch: 18 Loss: 1.5501535
Epoch: 19 Loss: 1.5202152
Epoch: 20 Loss: 1.527372
Epoch: 21 Loss: 1.4822496
Epoch: 22 Loss: 1.489616
Epoch: 23 Loss: 1.4626489
Epoch: 24 Loss: 1.4608938
Epoch: 25 Loss: 1.4408169
Epoch: 26 Loss: 1.4348143
Epoch: 27 Loss: 1.4152055
Epoch: 28 Loss: 1.4132502
Epoch: 29 Loss: 1.3974974
Epoch: 30 Loss: 1.3900427
Epoch: 31 Loss: 1.3814248
Epoch: 32 Loss: 1.3733743
Epoch: 33 Loss: 1.3626596
Epoch: 34 Loss: 1.3606967
Epoch: 35 Loss: 1.3360015
Epoch: 36 Loss: 1.3371056
Epoch: 37 Loss: 1.3250521
Epoch: 38 Loss: 1.3174871
Epoch: 39 Loss: 1.3089204


Epoch: 296 Loss: 0.5576919
Epoch: 297 Loss: 0.5387074
Epoch: 298 Loss: 0.55566114
Epoch: 299 Loss: 0.5368554
Epoch: 300 Loss: 0.54685545
Epoch: 301 Loss: 0.52805346
Epoch: 302 Loss: 0.54403114
Epoch: 303 Loss: 0.52084434
Epoch: 304 Loss: 0.5459362
Epoch: 305 Loss: 0.5126323
Epoch: 306 Loss: 0.5414714
Epoch: 307 Loss: 0.51358044
Epoch: 308 Loss: 0.53859466
Epoch: 309 Loss: 0.50913787
Epoch: 310 Loss: 0.5296135
Epoch: 311 Loss: 0.5120751
Epoch: 312 Loss: 0.5213061
Epoch: 313 Loss: 0.5087022
Epoch: 314 Loss: 0.5142618
Epoch: 315 Loss: 0.5104329
Epoch: 316 Loss: 0.5086236
Epoch: 317 Loss: 0.51157427
Epoch: 318 Loss: 0.50020814
Epoch: 319 Loss: 0.51356524
Epoch: 320 Loss: 0.49776554
Epoch: 321 Loss: 0.5126197
Epoch: 322 Loss: 0.4950677
Epoch: 323 Loss: 0.50605845
Epoch: 324 Loss: 0.48579085
Epoch: 325 Loss: 0.5030857
Epoch: 326 Loss: 0.47789574
Epoch: 327 Loss: 0.50495976
Epoch: 328 Loss: 0.47140902
Epoch: 329 Loss: 0.5022693
Epoch: 330 Loss: 0.47246888
Epoch: 331 Loss: 0.49542332
Epoch: 33

Epoch: 592 Loss: 0.17493868
Epoch: 593 Loss: 0.17808208
Epoch: 594 Loss: 0.17677662
Epoch: 595 Loss: 0.17722285
Epoch: 596 Loss: 0.17449537
Epoch: 597 Loss: 0.17462862
Epoch: 598 Loss: 0.16882235
Epoch: 599 Loss: 0.17361887
Epoch: 600 Loss: 0.16568232
Epoch: 601 Loss: 0.17155634
Epoch: 602 Loss: 0.16397777
Epoch: 603 Loss: 0.17044844
Epoch: 604 Loss: 0.16445577
Epoch: 605 Loss: 0.16896445
Epoch: 606 Loss: 0.16497312
Epoch: 607 Loss: 0.16754942
Epoch: 608 Loss: 0.16434968
Epoch: 609 Loss: 0.1646232
Epoch: 610 Loss: 0.16308656
Epoch: 611 Loss: 0.16399896
Epoch: 612 Loss: 0.16218852
Epoch: 613 Loss: 0.16211264
Epoch: 614 Loss: 0.16338794
Epoch: 615 Loss: 0.16029269
Epoch: 616 Loss: 0.16315033
Epoch: 617 Loss: 0.16304418
Epoch: 618 Loss: 0.16238347
Epoch: 619 Loss: 0.16056737
Epoch: 620 Loss: 0.15967633
Epoch: 621 Loss: 0.15488005
Epoch: 622 Loss: 0.15892062
Epoch: 623 Loss: 0.1518391
Epoch: 624 Loss: 0.15745565
Epoch: 625 Loss: 0.15052338
Epoch: 626 Loss: 0.15623373
Epoch: 627 Loss: 0.151

Epoch: 884 Loss: 0.062434673
Epoch: 885 Loss: 0.0645641
Epoch: 886 Loss: 0.061973143
Epoch: 887 Loss: 0.06400395
Epoch: 888 Loss: 0.062483482
Epoch: 889 Loss: 0.063645825
Epoch: 890 Loss: 0.06314191
Epoch: 891 Loss: 0.06471543
Epoch: 892 Loss: 0.063322276
Epoch: 893 Loss: 0.06369587
Epoch: 894 Loss: 0.062103145
Epoch: 895 Loss: 0.06119199
Epoch: 896 Loss: 0.061565664
Epoch: 897 Loss: 0.05966002
Epoch: 898 Loss: 0.0611718
Epoch: 899 Loss: 0.05896361
Epoch: 900 Loss: 0.060859315
Epoch: 901 Loss: 0.059194706
Epoch: 902 Loss: 0.061015792
Epoch: 903 Loss: 0.05934117
Epoch: 904 Loss: 0.06148715
Epoch: 905 Loss: 0.059059653
Epoch: 906 Loss: 0.060504086
Epoch: 907 Loss: 0.058295775
Epoch: 908 Loss: 0.06062986
Epoch: 909 Loss: 0.058095243
Epoch: 910 Loss: 0.059893563
Epoch: 911 Loss: 0.058651317
Epoch: 912 Loss: 0.05976746
Epoch: 913 Loss: 0.058925554
Epoch: 914 Loss: 0.060856745
Epoch: 915 Loss: 0.059271857
Epoch: 916 Loss: 0.059774555
Epoch: 917 Loss: 0.058174644
Epoch: 918 Loss: 0.05728114
E

In [7]:
# Print an Accuracy
acc = 0

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(len(testlist)):
        batch_data, batch_label = batch(testlist, 1)
        logit = sess.run(output, feed_dict = {X:batch_data})
        if np.argmax(logit[0]) == batch_label[0]:
            acc += 1
        else:
            print(logit[0], batch_label[0])
            
    print("Accuracy:", acc/len(testlist))

INFO:tensorflow:Restoring parameters from logs\model.ckpt-1000
[-0.12030726 -3.281833   -4.6532474   1.4550184 ] 0
[ -2.4777465   7.056954  -10.055142    2.309625 ] 0
[-1.2164823   8.837171   -0.66297483 -2.0606356 ] 0
[ 1.9857156 -5.870653   6.470988  -0.2034377] 0
[-1.8764936 -1.1147162 -5.9599423  7.5368237] 0
[-3.5484948  -7.1042027   0.22709276  7.767265  ] 0
[-5.098679  -4.30971    3.3988812  1.3152031] 0
[-0.05911385  3.4939508  -5.558946   -1.4235957 ] 0
[-0.36215955  3.6515214   5.703395   -3.472001  ] 0
[-7.8600597  6.2443237  7.1569676 -3.6156983] 0
[ 0.64147097 -1.5988799   4.9654655  -4.648388  ] 0
[ 1.9114863  4.4253507  5.9227247 -7.335736 ] 0
[ 4.9575505 -8.860134   6.7882423 -4.9683924] 0
[-2.877397    0.32375586  1.5115387  -0.21094956] 0
[-0.10113918  8.202822   -1.7576274  -2.8392599 ] 0
[-0.4883746  6.618279  -7.9138927  0.839384 ] 0
[-1.3141445  1.5968447  3.6116095 -8.722089 ] 0
[ 1.3349209  6.1164327  3.4531128 -6.0038385] 0
[ 3.5587907  0.5471276  8.966492  -6.

[-1.1286565   6.931317   -4.6676955  -0.20109098] 3
[ 5.0763054 -8.138518   1.6620226 -1.8426087] 3
[ 8.53099    7.6264663 -1.6522506 -8.886767 ] 3
[-5.594768   7.33219   -5.418709   3.5177734] 3
[ 1.975495    2.7416673   0.65969265 -6.9638367 ] 3
[ 4.843412    0.94500667  1.5853206  -3.1576502 ] 3
Accuracy: 0.2833333333333333
